In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import h5py

#model_file = "DL1_AntiKt4EMTopo_dropout" if dropout_enable else "DL1_AntiKt4EMTopo"
import models.rebuild_DL1 as DL1
from util import (DL1_score, plot_prob_score, plot_prob_score_from_model,
                  get_mean_score, momentum_space, efficiecy_mean_std)
    
def load_trained(model_weight):
    import models.binbin_model as binbin_model
    test_model = binbin_model.DL1_model(InputShape=44, training=False)
    test_model_Dropout = binbin_model.DL1_model(InputShape=44, training=True)
    test_model.load_weights(model_weight)
    test_model_Dropout.load_weights(model_weight)
    return test_model, test_model_Dropout

def data_category(data, label,category='b'):
    #1 is b, 2 is c, 4 is light
    label_binary = {'b':2, 'c':1, 'l':0}
    label_index = label_binary[category]
    label_filter = (label[:,label_index]==1)
    print(label_filter)
    return data[label_filter], label[label_filter]
    
    
#get test dataset
hf = h5py.File("/Users/abletimin/cernbox/b-tagg/btagging-ml_tutorial_files/MC16d_ttbar-test-validation_sample-NN.h5", 'r')
X_test, Y_test = hf['X_test'][:], hf['Y_test'][:]
hf.close()

X_test, Y_test = data_category(X_test, Y_test,category='b')


[ True  True  True ... False  True False]


## import model


In [2]:
trained_weight="/Users/abletimin/cernbox/b-tagg/trainedModel/DL1r_hybrid_50epochs_3600batch.h5"
model, model_dropout =load_trained(model_weight=trained_weight)

In [ ]:
scores = get_mean_score(X_test, model_dropout, 5000)

/Users/abletimin/work/ATLAS/b-tagg/DL1_model/util/tools.py:19: RuntimeWarning: overflow encountered in log
  return np.log(pb/(0.08*pc + 0.92*pl))
